In [6]:
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt
from PIL import Image

In [7]:
# Load pointcloud from file and save its point position and color as two numpy arrays
pcd_path = "Pointclouds/demo_ (1).pcd"
pcd_load = o3d.io.read_point_cloud(pcd_path, format='pcd')
pcd_points = np.asarray(pcd_load.points)
pcd_colors_orig = np.asarray(pcd_load.colors)

In [8]:
# Optional: View the loaded pointcloud
o3d.visualization.draw_geometries([pcd_load],
                                  zoom=0.5,
                                  front=[ 0, 0, -1],
                                  lookat=[ 0, 0, .3],
                                  up=[ 0, -1, 0])

In [ ]:
# Load and view leaf mask
mask_image = Image.open('Images/aggrigated_masks (1).png')
mask_array = np.asarray(mask_image)[:, :, 0:3]
plt.imshow(mask_array)

In [10]:
# Format mask array to be compatible with o3d's color array setup
reshape_mask = np.reshape(mask_array, (1555200, 3))
mask_norm = reshape_mask / 255

In [11]:
# Use leaf mask to remove all non-leaf portions of the pointcloud
pcd_colors = np.asarray(pcd_load.colors)
index = np.argwhere(reshape_mask/255==mask_norm[0])
pcd_colors = np.delete(pcd_colors, index, 0)
pcd_points = np.delete(pcd_points, index, 0)
pcd_colors_orig = np.delete(pcd_colors_orig, index, 0)

In [12]:
# Generate the new masked pointcloud
pcd = o3d.geometry.PointCloud()
pcd.colors = o3d.utility.Vector3dVector(pcd_colors_orig)
pcd.points = o3d.utility.Vector3dVector(pcd_points)

In [13]:
# Clean up point cloud by removing outliers.
# Since we are targeting points with a neighbor count lower than the average, this will remove
# the areas of the point cloud with a very high depth slope. This removes wildly wrong depth artifacts
# And removes the steep transition between different depths that the smooth output of the RAFT model produced.
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=100, std_ratio=1.0)
inlier_cloud = pcd.select_by_index(ind)
outlier_cloud = pcd.select_by_index(ind, invert=True)
outlier_cloud.paint_uniform_color([1, 0, 0])

PointCloud with 7350 points.

In [14]:
# Optional: View the cleaned up point cloud. Small clusters still need to be removed
o3d.visualization.draw_geometries([inlier_cloud],
                                  zoom=0.5,
                                  front=[ 0, 0, -1],
                                  lookat=[ 0, 0, .3],
                                  up=[ 0, -1, 0])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
